In [21]:
import numpy as np
import random

num_entities = 10
num_validators = 500000
num_slots = 300000
fixed_reward = 0.025
fixed_mev_reward = 0.1
split_threshold = 32

# Create validator_entity_mapping and initialize rewards
validator_entity_mapping = [i % num_entities for i in range(num_validators)]
validator_rewards = np.zeros(num_validators)
entity_rewards = np.zeros(num_entities)

# Create lists of validators per entity
validators_per_entity = [[] for _ in range(num_entities)]
for validator_idx, entity in enumerate(validator_entity_mapping):
    validators_per_entity[entity].append(validator_idx)

# Run the simulation
for _ in range(num_slots):
    # Select a random validator
    chosen_validator = random.randrange(num_validators)

    # Assign rewards
    block_reward = fixed_reward
    mev_reward = fixed_mev_reward
    total_reward = block_reward + mev_reward

    validator_rewards[chosen_validator] += total_reward
    entity_rewards[validator_entity_mapping[chosen_validator]] += total_reward

    # Check if we need to spawn a new validator for any entity
    for entity_idx in range(num_entities):
        if entity_rewards[entity_idx] >= split_threshold:
            # Spawn a new validator for this entity
            num_validators += 1
            validator_entity_mapping.append(entity_idx)
            validator_rewards = np.append(validator_rewards, 0)
            validators_per_entity[entity_idx].append(num_validators - 1)

            # Deduct the rewards used to spawn the new validator from the existing validators
            to_deduct = split_threshold
            for validator_idx in validators_per_entity[entity_idx]:
                if to_deduct > 0:
                    deduct_amount = min(validator_rewards[validator_idx], to_deduct)
                    validator_rewards[validator_idx] -= deduct_amount
                    entity_rewards[entity_idx] -= deduct_amount
                    to_deduct -= deduct_amount

# Print the top and bottom 20 validator stakes
sorted_indices = np.argsort(validator_rewards)
print("Top 20 validator stakes:")
for idx in sorted_indices[-20:]:
    print(f"Validator {idx}: {validator_rewards[idx]}")

print("Bottom 20 validator stakes:")
for idx in sorted_indices[:20]:
    print(f"Validator {idx}: {validator_rewards[idx]}")

# Count validators per entity
validators_count_per_entity = [len(validators) for validators in validators_per_entity]

# Calculate the total value each entity controls
entity_values = np.array(validators_count_per_entity) * 32 + entity_rewards

# Print the results
print("Validators per entity:")
for i in range(num_entities):
    print(f"Entity {i}: {validators_count_per_entity[i]}")

print("Total value each entity controls:")
for i in range(num_entities):
    print(f"Entity {i}: {entity_values[i]}")


Top 20 validator stakes:
Validator 90340: 0.125
Validator 316812: 0.125
Validator 59366: 0.125
Validator 97495: 0.125
Validator 332055: 0.125
Validator 465538: 0.125
Validator 362651: 0.125
Validator 362640: 0.125
Validator 371553: 0.125
Validator 362630: 0.125
Validator 332036: 0.125
Validator 425411: 0.125
Validator 263056: 0.125
Validator 168608: 0.125
Validator 258355: 0.125
Validator 20008: 0.25
Validator 438643: 0.25
Validator 306933: 0.25
Validator 348938: 0.25
Validator 336776: 0.25
Bottom 20 validator stakes:
Validator 0: 0.0
Validator 333863: 0.0
Validator 333862: 0.0
Validator 333861: 0.0
Validator 333860: 0.0
Validator 333859: 0.0
Validator 333858: 0.0
Validator 333857: 0.0
Validator 333856: 0.0
Validator 333855: 0.0
Validator 333854: 0.0
Validator 333853: 0.0
Validator 333852: 0.0
Validator 333864: 0.0
Validator 333851: 0.0
Validator 333849: 0.0
Validator 333848: 0.0
Validator 333847: 0.0
Validator 333846: 0.0
Validator 333845: 0.0
Validators per entity:
Entity 0: 50116
En